In [23]:
import json
import pandas as pd
from collections import defaultdict
from glob import glob
from os import path
from datetime import datetime

In [38]:
FILES_DIR = '<DIR>' # directory with ogg files from archive (files from Bond)

In [25]:
ogg_files = glob(path.join(FILES_DIR, 'files/*'))

In [26]:
files = [path.basename(x) for x in ogg_files]

In [27]:
with open('../youtube-api/videos_list_yutt.json') as f:
    videos_list_yutt = json.loads(f.read())

In [28]:
manual_mapping = {
    '20170930_The \'Ball of Honey\' Discourse - Madhupindika Sutta (MN 18).ogg': 'https://youtu.be/nlSpMukrJzs',
    '20111111_Ask A Monk - Meditating 24_7.ogg': 'https://youtu.be/AA8tL9eTUxE',
    '20110806_Dhammapada Verse One - Mind Precedes All Things.ogg': 'https://youtu.be/lZZiE-EofAE',
    '20100809_Ask A Monk - \'That Place Inside\' and Issues from Childhood.ogg': 'https://youtu.be/e4I_Nx5vDmY',
    '20100803_Ask A Monk - Meaning of \'Mindfulness\' and \'Clear Thought\'.ogg': 'https://youtu.be/zAEubYyR43g',
    '20100726_LIAD - 9_30 PM - Ask A Monk.ogg': 'https://youtu.be/OyMWmLAFtJQ',
    '20100726_LIAD - 10_30 AM - Alms Round 51 of 66.ogg': 'https://youtu.be/KF0FFBjGVlw',
    '20100725_LIAD - 10_30 AM Almsround 54 of 66.ogg': 'https://youtu.be/M_O7rZ3MSNQ',
    '20100725_LIAD - 10_30 AM Almsround 25 of 66.ogg': 'https://youtu.be/t_XP_xbs_ig',
    '20100725_LIAD - 10_30 AM - Almsround 8 of 66.ogg': 'https://youtu.be/oYsdWxsZ8Fc',
    '20100725_LIAD - 10_30 AM - Almsround 4 of 66.ogg': 'https://youtu.be/q-sfslbV_OI',
    '20100725_LIAD - 10_30 AM - Almsround 12 of 66.ogg': 'https://youtu.be/lO8w4Zn_MI0',
    '20100724_LIAD - 4_30 AM - Chanting 1 of 2.ogg': 'https://youtu.be/zU33ID0PBlc',
    '20100726_LIAD - 10_30 AM - Alms Round 15 of 66.ogg': 'https://youtu.be/9mc666t8S90',
    '20100725_LIAD - 10_30 AM Almsround 59 of 66.ogg': 'https://youtu.be/egN4M78T8og',
    '20100725_LIAD - 10_30 AM Almsround 57 of 66.ogg': 'https://youtu.be/0q7mv_avQwo',
    '20100502_Free \'How To Meditate\' Two-DVD set.ogg': 'https://youtu.be/2HiktzLcODw'
}

In [29]:
found = set()
canditates = defaultdict(list)

for file in files:
    dt_file = datetime.strptime(file[:8], "%Y%m%d")
    name = file[9:-4]
    
    if file in manual_mapping:
        found.add(file)
        continue

    for video in videos_list_yutt:
        dt = datetime.strptime(video['snippet']['publishedAt'][:10], "%Y-%m-%d")
        
        dt_diff = (dt_file - dt).days
        
        if abs(dt_diff) > 1:
            continue
            
        canditates[file].append(video)
        
        if video['snippet']['title'].replace('?', '') in [name, name.replace(' -', ':', 1).replace('_', ' ')]:
            found.add(file)
            
            if file in canditates:
                del canditates[file]
            
            break

In [30]:
not_found = set(files) - found
not_found = sorted(not_found)[::-1]

In [31]:
len(not_found)

91

In [32]:
with open('files_not_found.json', 'w') as f:
    f.write(json.dumps(not_found))

In [41]:
pd.DataFrame(not_found).to_excel('files_not_found.xlsx')

In [34]:
#canditate_list = []

#for file in not_found:
#    if not file in canditates:
#        canditate_list.append({
#            'filename': file
#        })
#    else:
#        for video in canditates[file]:
#            canditate_list.append({
#                'filename': file,
#                'candidateTitle': video['snippet']['title'],
#                'candidateLink': 'https://youtu.be/' + video['id']
#            })
#df = pd.DataFrame(canditate_list)
#df.to_excel('not_found_candidates.xlsx')

In [35]:
#import subprocess
#import shlex
#import re
#import isodate
#from datetime import timedelta

In [36]:
#def get_ogg_lenght(file):
#    ffprobe = subprocess.check_output('ffprobe ' + shlex.quote(file), stderr=subprocess.STDOUT, shell=True)
#    duration = re.search(r'Duration: ([^,]+)', str(ffprobe)).group(1)
#    return timedelta(hours=int(duration[:2]), minutes=int(duration[3:5]), seconds=int(float(duration[6:])))

In [39]:
#candidates = defaultdict(list)
#canditate_list = []

#for file in not_found:
#    duration_file = get_ogg_lenght(path.join('<DIR>', file))
#    
#    for video in videos_list_yutt:
#       duration = isodate.parse_duration(video['contentDetails']['duration'])
#        
#        diff = duration - duration_file if duration >= duration_file else duration_file - duration
#        
#        if diff < timedelta(seconds=3):
#            canditate_list.append({
#                'filename': file,
#                'candidateTitle': video['snippet']['title'],
#                'candidateDate': video['snippet']['publishedAt'],
#                'candidateLink': 'https://youtu.be/' + video['id']
#            })
#            candidates[file].append(video)
#pd.DataFrame(canditate_list).to_excel('test.xlsx')